<a href="https://colab.research.google.com/github/Kiddo-JB/DE_ETL_Project/blob/main/DE_ETL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL PROJECT

In [1]:
# Importing Libraries
import pandas as pd
import json
import xml.etree.ElementTree as ET
from datetime import datetime

In [2]:
# Define file paths
LOG_FILE = 'log_file.txt'
TRANSFORMED_DATA = 'transformed_data.csv'

In [4]:
# Write a log message to a text file.
def log_message(message):
    with open(LOG_FILE, 'a') as log_file:
      log_file.write(f"{datetime.now()} - {message}\n")

# Extract Methods

In [9]:
def extract_csv(file_path):
   # Extract data from a CSV file.
    log_message(f"Extracting data from CSV file: {file_path}")
    return pd.read_csv(file_path)

def extract_json(file_path):
    # Extract data from a JSON file.
    log_message(f"Extracting data from JSON file: {file_path}")
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]

    return pd.json_normalize(data)


def extract_xml(file_path):
   #Extract data from an XML file.
    log_message(f"Extracting data from XML file: {file_path}")
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = [{elem.tag: elem.text for elem in child} for child in root]
    return pd.DataFrame(data)

def extract_data(file_paths):
    #Master function to extract data based on file type.
    log_message("Starting data extraction")
    all_data = []
    for file_path in file_paths:
        if file_path.endswith('.csv'):
            all_data.append(extract_csv(file_path))
        elif file_path.endswith('.json'):
            all_data.append(extract_json(file_path))
        elif file_path.endswith('.xml'):
            all_data.append(extract_xml(file_path))
        else:
            log_message(f"Unsupported file format: {file_path}")
    combined_data = pd.concat(all_data).drop_duplicates()
    log_message("Data extraction completed")
    return combined_data

# Transformation Method


In [10]:
def transform_data(df):
    #Transform the data (e.g., unit conversions, cleaning).
    log_message("Starting data transformation")
    # Convert heights to meters and weights to kilograms
    for col in df.columns:
        if col.lower() =='height':
            df['height_in_meters'] = (df[col].astype(float) * 0.0254).round(2)
            df.drop(columns=[col], inplace=True)
        if col.lower() =='weight':
            df['weight_in_kilograms'] = (df[col].astype(float) * 0.453592).round(2)
            df.drop(columns=[col], inplace=True)
    log_message("Data transformation completed")
    return df

# Loading Method

In [11]:
def load_data(df, output_path):
    #Load the transformed data into a CSV file.
    log_message(f"Loading data into CSV file: {output_path}")
    df.to_csv(output_path)
    log_message(f"Data successfully loaded into {output_path}")


# Main Method

In [12]:
def etl_process(file_paths, output_path):
    # ETL process: Extract, Transform, Load.
    log_message("Starting ETL process")

    # Extract
    extracted_data = extract_data(file_paths)

    # Transform
    transformed_data = transform_data(extracted_data)

    # Load
    load_data(transformed_data, output_path)

    log_message("ETL process completed")


file_paths = ['./source1.csv', './source1.json', './source1.xml','./source2.csv', './source2.json', './source2.xml','./source3.csv', './source3.json', './source3.xml']
etl_process(file_paths, TRANSFORMED_DATA)